In [1]:
import pymysql
from pymysql.err import MySQLError
import time

def get_username_link(post_id, conn):
    """ get a single user link profile, ima profile and username from the db """
    
    try:
        with conn.cursor() as cursor:
            # SQL query to fetch the post link by ID
            sql = "SELECT username, username_link FROM posts WHERE id = %s"
            cursor.execute(sql, (post_id,))       
            result = cursor.fetchone()

            if result:
                return result
            else:
                return None
        
    except MySQLError as e:
        print('Mysql Connection Error:', e)

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def start_driver():
    """ Initialize Chrome driver """
    
    # Chrome driver PATH
    CHROMEDRIVER_PATH = r"C:\Users\lucal\.wdm\drivers\chromedriver\win64\131.0.6778.85\chromedriver-win32/chromedriver.exe"
    
    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH))  
    time.sleep(2)
    
    return driver

In [3]:
# Import the By class for locating elements
from selenium.webdriver.common.by import By
# Import wait and EC to form an implicit wait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# timeout exception
from selenium.common.exceptions import TimeoutException

def accept_cookies(driver):
    """ accept coookies """

    # CSS SELECTOR
    ACCEPT_COOKIE_SELECTOR = 'body > div.__fb-dark-mode.x1n2onr6.xzkaem6 > div:nth-child(2) > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div.x1uvtmcs.x4k7w5x.x1h91t0o.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1n2onr6.x1qrby5j.x1jfb8zj > div > div > div > div.x6s0dn4.x78zum5.xl56j7k.x1n2onr6.x1vjfegm > div > div > div > div.x6bh95i.x13fuv20.x178xt8z.x1p5oq8j.xxbr6pl.xwxc41k.xbbxn1n > div > div.x1i10hfl.x1ypdohk.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xexx8yu.x18d9i69.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1q0g3np.x1lku1pv.x1a2a7pz.x6s0dn4.x1a2cdl4.xnhgr82.x1qt0ttw.xgk8upj.x9f619.x3nfvp2.x1s688f.x90ne7k.xl56j7k.x193iq5w.x1swvt13.x1pi30zi.x12w9bfk.x1g2r6go.x11xpdln.xz4gly6.x87ps6o.xuxw1ft.x19kf12q.x111bo7f.x1vmvj1k.x45e8q.x3tjt7u.x35z2i1.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x178xt8z.xm81vs4.xso031l.xy80clv.xteu7em.x11r8ahe.x1iyjqo2.x15x72sd > div'
    # XPATH
    ACCEPT_COOKIE_XPATH = "//div[contains(text(), 'Allow all cookies')]"

    # TIMEOUT (15 seconds)
    TIMEOUT = 15
    
    try:
        # implicit wait
        button = WebDriverWait(driver, TIMEOUT).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ACCEPT_COOKIE_SELECTOR)) # with CSS_SELCTOR
        )
        # click to accept cookies
        button.click()
        print("CSS cookies button click")
    
    except TimeoutException:
        try: 
            button = WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.XPATH, ACCEPT_COOKIE_XPATH)) # with XPATH
            )
            button.click()
            print("XPTAH cookies button click")

        except TimeoutException:    
            print("ERROR: Cookies button missing")
            driver.quit() # cleaner termination ensuring a clean state for future executions

In [4]:
#XPATH
""" 
1ST PROFILE:
1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[1]/div/div/div/div
2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[2]/div/div/div/div
3RD POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[3]/div/div/div/div

2ND PROFILE
1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[1]/div/div[1]/div/div[2] (1/2 fixed post)
           //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[1]/div/div[2]/div/div    (2/2)
2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[2]/div/div/div/div

**************************************

POST LINK XPATH : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[2]/div/div/div/div/div[2]/div/div[1]/div/span[2]/a
          
"""

POST_LINK_REL_XPATH     = './/div[2]/div/div[1]/div/span[2]/a'

In [5]:
# class to handle each post
class Post:
    def __init__(self, username, username_link, post_link):
        self._username = username
        self._username_link = username_link
        self._post_link = post_link
    
    #getter
    @property
    def single_post(self):
        return {
            "username": self._username,
            "username_link": self._username_link,
            "post_link": self._post_link
            }

In [6]:
from selenium.common.exceptions import NoSuchElementException

def grab_post(driver, post_number, username, username_link):
    """ Grab post and return: image profile link, username, username link and post link """
    
    # Post XPATH
    POST_XPATH = f'//*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[4]/div/div[1]/div[{post_number}]/div/div/div/div'
  
    # retrieves a specific post on the page
    post = driver.find_element(By.XPATH, POST_XPATH)

    if post:
        # post height
        post_height = post.size['height']
        
        # fecth post link
        try:
            # post link
            post_link = post.find_element(By.XPATH, POST_LINK_REL_XPATH).get_attribute('href')

            # new instance of the class Post
            new_post_obj = Post(username, username_link, post_link)
    
            # create dictonary
            new_post = new_post_obj.single_post
        
        # incomplete post
        except NoSuchElementException as e:
            print("grab_post function NoSuchElementException:", e) 
    
        # other exception
        except Exception as e:
            print("Exception in grab_post function:", e)
            
        # return the new post collected   
        return (new_post, post_height)
    
    else:
        print("POST wrong XPATH")        

In [7]:
def scroll(driver, px):
    """Scroll down by the height of a post"""
    
    js = f"window.scrollBy(0, {px});"
    driver.execute_script(js)

In [8]:
import pymysql
from pymysql.err import MySQLError

def manual_migration(conn):
    """ Create the 'post_users' table only if it doesn't exist """

    try:
        with conn.cursor() as cursor:
            # SQL query to create the posts table with Unicode support
            create_table_query = """
                CREATE TABLE IF NOT EXISTS post_users (
                    id BIGINT AUTO_INCREMENT PRIMARY KEY,
                    username VARCHAR(255) NOT NULL,
                    img_profile TEXT NOT NULL,
                    user_link TEXT NOT NULL,
                    bio TEXT,
                    followers DOUBLE,
                    UNIQUE (username),
                    created TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
                );
            """
                    
            # execute query
            cursor.execute(create_table_query)

            # Commit the transaction
            conn.commit()

    except MySQLError as e:
        print('Mysql Error:', e) 

In [9]:
def save_to_db(conn, data):
    """Save data into MySQL with duplicate handling using INSERT IGNORE """

    try:
        with conn.cursor() as cursor:
            # Create an SQL insert statement with INSERT IGNORE to skip duplicates
            sql = """
                INSERT IGNORE INTO posts (username, username_link, post_link) 
                VALUES (%s, %s, %s)
                """
            
            # Prepare the data as a list of tuples
            values = [(post['username'], post['username_link'], post['post_link']) for post in data]

            try:
                # Execute the insert statement for all data at once
                rows_inserted = cursor.executemany(sql, values)
                
                # Commit the transaction
                conn.commit()
                print(f'{rows_inserted} rows saved in "posts" database correctly')
            
            except MySQLError as e:
                print('Mysql insert query Error:', e)
    
    except MySQLError as e:
        print('Mysql Error:', e)

In [10]:
def scroll_to_bottom(driver, conn, username, user_link):
    """ auto-scroll to the bottom of the page and save posts to db """
  
    # post number to retrieve
    post_number = 1
    # number of posts to be saved together
    post_no_to_save = 10
    
    # list of posts
    list_to_push_to_db = []
    
    old_position = driver.execute_script("return window.pageYOffset;")
    px_to_scroll = 0
    
    while True:

        # Wait for the page to load
        time.sleep(3)

        try:
            # retrieve post
            post, post_height = grab_post(driver, post_number, username, user_link)

             # Add post link to list only if valid
            if post:
                list_to_push_to_db.append(post)
                print(f"post number {post_number} added: {post}")

            # save to db in batches
            if len(list_to_push_to_db)>=post_no_to_save:
                save_to_db(conn, list_to_push_to_db)
                list_to_push_to_db = [] # free memory

            # Scroll a little less than the height of the post
            px_to_scroll = max(20, abs(post_height - 20))
            
            # update post position
            post_number += 1
 

        # post not found
        except NoSuchElementException:
            print(f"Post number {post_number} not found")
            continue

        
        except Exception as e:
            print(f"Post number {post_number} Error: {e}")
            # skip post
            post_number += 1
            continue

        
        finally:
            # scroll down
            print("scroll down")
            scroll(driver, px_to_scroll)
            
            # update position
            new_position = driver.execute_script("return window.pageYOffset;")

            # Exit the loop if the page hasn't scrolled, meaning end of page
            if new_position == old_position:
                print('End of Page. Taking a new user...')
                break  
            
            # update new position
            old_position = new_position 
            
    # Save remaining posts to the database
    if list_to_push_to_db:
        print(f"Saving remaining {len(list_to_push_to_db)} posts to the database...")
        save_to_db(conn, list_to_push_to_db)

In [11]:
def check_table_and_post(post_id, wait_time):
    """ Check if the table exists and contains a the post_id==1 """
    table_name = "posts"
    retries = 0
    MAX_RETRIES = 20

    while retries < MAX_RETRIES:
        try:
            # conncetion to db
            conn = pymysql.connect(
                host='localhost',
                user='mysql',
                password='mysql',
                database='mysql',
                port=3306
            )
            
            with conn.cursor() as cursor:
                # Check if the table "posts" exists
                cursor.execute(f"SHOW TABLES LIKE '{table_name}';")
                table_exists = cursor.fetchone()
                
                if not table_exists:
                    print(f"Table '{table_name}' does not exist. Retrying in {wait_time} seconds...")
                    retries += 1
                    time.sleep(wait_time)
                    continue
                
                # Check if the table contains the specific post_id 1
                cursor.execute(f"SELECT 1 FROM {table_name} WHERE id = %s LIMIT 1;", (post_id,))
                post_exists = cursor.fetchone()
                
                if not post_exists:
                    print(f"Post with id={post_id} not found in table '{table_name}'. Retrying in {wait_time} seconds...")
                    retries += 1
                    time.sleep(wait_time)
                    continue
                
                # Table exists and post is present
                print(f"Table '{table_name}' exists and contains post with id={post_id}.")
                return True
        
        except pymysql.MySQLError as e:
            print(f"Database error: {e}. Retrying in {wait_time} seconds...")
            retries += 1
            time.sleep(wait_time)

        finally:
            conn.close()
    
    print(f"Exceeded maximum retries ({max_retries}). Exiting.")
    return False

In [12]:
# class to save user info

class UserInfo:
    def __init__(self, username, img_profile, user_link, bio, followers):
        self.username = username
        self.img_profile = img_profile
        self.user_link = user_link
        self.bio = bio
        self.followers = followers

    @property
    def info(self):
        return {
            "username" : self.username,
            "img_profile" : self.img_profile,
            "user_link" : self.user_link,
            "bio" : self.bio,
            "followers" : self.followers
        }

In [13]:
# XPATH User Info
"""
1ST USER BIO : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div[2]/span
2ND USER BIO : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div[2]/span

*************************

1ST USER FOLLOWERS : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div[3]/div[1]/div/span/span
2ND USER FOLLOWERS : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div[3]/div[1]/div/span/span
                     
"""

BIO_XPATH = '//*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div[2]/span'
FOLLOWERS_XPATH = '//*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div[3]/div[1]/a/div/span/span'
PIC_XPATH = '//*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div[1]/div[2]/div/div/div/img'                  

In [14]:
def get_bio(driver):
    """ return user bio """

    try:
        return driver.find_element(By.XPATH, BIO_XPATH).text
        
    except NoSuchElementException:
        print("Bio not found!")
        return None

In [15]:
def get_followers(driver):
    """ return user followers"""

    try:
        return driver.find_element(By.XPATH, FOLLOWERS_XPATH).text

    except NoSuchElementException:
        print("Followers not found!")
        return None

In [16]:
def get_profile_pic(driver):
    """ get and return profile pic of a user """
    try:
        return driver.find_element(By.XPATH, PIC_XPATH).get_attribute("src")
    
    except NoSuchElementException:
        print("Profile pic not found!")
        return None    

In [17]:
def fetch_user_info(driver, username, user_link):
    """ catch user profile important info and return a dict """   

    time.sleep(2)
    
    # get user bio
    bio = get_bio(driver)
    
    # get user followers
    followers = float(get_followers(driver))

    # get user profile pic
    img_profile = get_profile_pic(driver)

    # buld class
    user = UserInfo(username, img_profile, user_link, bio, followers)
    
    return user.info

In [18]:
def save_user(conn, user_info):
    """ save user info in "post_users" table """
    try:
        with conn.cursor() as cursor:
            # SQL query to insert or update user info
            query = """
                INSERT INTO post_users (username, img_profile, user_link, bio, followers)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    img_profile = VALUES(img_profile),
                    user_link = VALUES(user_link),
                    bio = VALUES(bio),
                    followers = VALUES(followers);
            """
            
            # Extract values from user_info dictionary
            values = (
                user_info["username"],
                user_info["img_profile"],
                user_info["user_link"],
                user_info["bio"],
                user_info["followers"]
            )
            
            # Execute the query
            cursor.execute(query, values)
            
            # Commit the transaction
            conn.commit()
            print(f"User '{user_info['username']}' saved successfully.")

    except MySQLError as e:
        print(f"Error saving user '{user_info['username']}': {e}")

In [19]:
from selenium.common.exceptions import InvalidSessionIdException

def close_session(driver):
    """ close active browser tab """
    
    try:
        driver.close() 
    except InvalidSessionIdException:
        print("Browser tab already closed!")

In [20]:
def main():
    """ main function """
    
    # first user to scrape
    post_id = 1


    try:
        # conncetion to db
        conn = pymysql.connect(
            host='localhost',
            user='mysql',
            password='mysql',
            database='mysql',
            port=3306
        )

        # Ensure the table "posts" exists and contains the post_id == 1
        if not check_table_and_post(post_id, 60):
            print("Required table or post not found. Exiting.")
            return
        
        # posts_info table creation
        manual_migration(conn)
        
        # start driver
        driver = start_driver()
        
        while True:
            # Attempt to get the user link
            (username, user_link) = get_username_link(post_id, conn)

            if user_link:
                print("\nUser Profile:", user_link)

                # open user's profile page
                driver.get(user_link)

                # first user
                if post_id == 1:
                    # accept cookies only once
                    accept_cookies(driver)  
                    
                # fetch user info
                user_info = fetch_user_info(driver, username, user_link)
                
                # save user info
                save_user(conn, user_info)

 
                # scroll page, fetch posts and add to the list of post to save in the db
                scroll_to_bottom(driver, conn, username, user_link)

  
                time.sleep(5)
            
            else:
                print(f"\nPost id {post_id} not found in db!")
                
            # Increment post_id to fetch the next post
            post_id += 1
                
    
    except Exception as e:
        print('Main Error:', e)
   
    finally:
        close_session(driver)
        if conn:
            conn.close()

In [22]:
############################
####### RUN SCRAPER ########
############################
main()

Table 'posts' exists and contains post with id=1.

User Profile: https://www.threads.net/@extratv
CSS cookies button click
User 'extratv' saved successfully.
post number 1 added: {'username': 'extratv', 'username_link': 'https://www.threads.net/@extratv', 'post_link': 'https://www.threads.net/@extratv/post/DDiXSJXy-VI'}
scroll down
post number 2 added: {'username': 'extratv', 'username_link': 'https://www.threads.net/@extratv', 'post_link': 'https://www.threads.net/@extratv/post/DDh1FePyMgS'}
scroll down
post number 3 added: {'username': 'extratv', 'username_link': 'https://www.threads.net/@extratv', 'post_link': 'https://www.threads.net/@extratv/post/DDhq02nPssI'}
scroll down
post number 4 added: {'username': 'extratv', 'username_link': 'https://www.threads.net/@extratv', 'post_link': 'https://www.threads.net/@extratv/post/DDhockrvjZO'}
scroll down
post number 5 added: {'username': 'extratv', 'username_link': 'https://www.threads.net/@extratv', 'post_link': 'https://www.threads.net/@e

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.140)
Stacktrace:
	GetHandleVerifier [0x00B533E3+25059]
	(No symbol) [0x00ADCDE4]
	(No symbol) [0x009BBEC3]
	(No symbol) [0x009AB21E]
	(No symbol) [0x009ABF8D]
	(No symbol) [0x009BC3F2]
	(No symbol) [0x0099D555]
	(No symbol) [0x00A2DAEE]
	(No symbol) [0x00A21EC3]
	(No symbol) [0x009F3FAC]
	(No symbol) [0x009F4F3D]
	GetHandleVerifier [0x00E45543+3113795]
	GetHandleVerifier [0x00E5A20A+3198986]
	GetHandleVerifier [0x00E529E2+3168226]
	GetHandleVerifier [0x00BF3250+680016]
	(No symbol) [0x00AE572D]
	(No symbol) [0x00AE29D8]
	(No symbol) [0x00AE2B75]
	(No symbol) [0x00AD57D0]
	BaseThreadInitThunk [0x75CB5D49+25]
	RtlInitializeExceptionChain [0x7743CEBB+107]
	RtlGetAppContainerNamedObjectPath [0x7743CE41+561]
